<a href="https://colab.research.google.com/github/Boingabu/programacion_avanzada/blob/main/Copia_de_Taller_optimizaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
!git config --global user.email "tomastuberquia32@gmail.com"
!git config --global user.name "Boingabu"

In [ ]:
!git clone https://github.com/Boingabu/programacion_avanzada.git

Cloning into 'programacion_avanzada'...


In [ ]:
!ls

programacion_avanzada  sample_data


In [ ]:
!cp /content/Taller_optimizacion/content/programacion_avanzada/Taller_optimizacion.ipynb/content/algoritmos-optimizacion/

cp: missing destination file operand after '/content/Taller_optimizacion/content/programacion_avanzada/Taller_optimizacion.ipynb/content/algoritmos-optimizacion/'
Try 'cp --help' for more information.


In [ ]:
%cd /content/programacion_avanzada
!git add programacion_avanzada.iy
!git commit -m "Subiendo archivo desde Colab"
!git push origin main

/content/programacion_avanzada
fatal: pathspec 'programacion_avanzada.iy' did not match any files
On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Boingabu/programacion_avanzada.git'


problema #1

In [ ]:
def mochila(capacidad, pesos, valores, n):
    # Creamos una tabla para almacenar los valores máximos
    K = [[0 for _ in range(capacidad + 1)] for _ in range(n + 1)]

    # Construimos la tabla K[][] de abajo hacia arriba
    for i in range(n + 1):
        for w in range(capacidad + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif pesos[i-1] <= w:
                K[i][w] = max(valores[i-1] + K[i-1][w-pesos[i-1]], K[i-1][w])
            else:
                K[i][w] = K[i-1][w]

    # Reconstruimos la solución para saber qué objetos se seleccionaron
    res = K[n][capacidad]
    w = capacidad
    objetos_seleccionados = []

    for i in range(n, 0, -1):
        if res <= 0:
            break
        if res == K[i-1][w]:
            continue
        else:
            objetos_seleccionados.append(i)
            res -= valores[i-1]
            w -= pesos[i-1]

    return K[n][capacidad], sorted(objetos_seleccionados)

# Casos de prueba
casos = [
    {"pesos": [2, 3, 4, 5], "valores": [3, 4, 5, 6], "capacidad": 5},
    {"pesos": [10, 20, 30], "valores": [60, 100, 120], "capacidad": 50},
    {"pesos": [1, 4, 3, 1], "valores": [1500, 3000, 2000, 2000], "capacidad": 5},
    {"pesos": [2, 3, 4, 5, 9], "valores": [3, 4, 5, 8, 10], "capacidad": 20},
    {"pesos": [1, 2, 3, 4], "valores": [10, 20, 30, 40], "capacidad": 4}
]

for i, caso in enumerate(casos, 1):
    valor, objetos = mochila(caso["capacidad"], caso["pesos"], caso["valores"], len(caso["pesos"]))
    print(f"Caso de Prueba {i}")
    print(f"Objetos seleccionados: {[f'Objeto {o}' for o in objetos]}")
    print(f"Valor total: {valor}\n")

Caso de Prueba 1
Objetos seleccionados: ['Objeto 1', 'Objeto 2']
Valor total: 7

Caso de Prueba 2
Objetos seleccionados: ['Objeto 2', 'Objeto 3']
Valor total: 220

Caso de Prueba 3
Objetos seleccionados: ['Objeto 1', 'Objeto 3', 'Objeto 4']
Valor total: 5500

Caso de Prueba 4
Objetos seleccionados: ['Objeto 1', 'Objeto 3', 'Objeto 4', 'Objeto 5']
Valor total: 26

Caso de Prueba 5
Objetos seleccionados: ['Objeto 1', 'Objeto 3']
Valor total: 40



In [ ]:
import heapq
from collections import defaultdict

def dijkstra_con_rutas(grafo, inicio, fin):
    # Diccionario para almacenar las distancias mínimas
    distancias = {nodo: float('inf') for nodo in grafo}
    distancias[inicio] = 0

    # Diccionario para almacenar el número de arcos
    arcos = {nodo: float('inf') for nodo in grafo}
    arcos[inicio] = 0

    # Diccionario para almacenar las rutas
    rutas = defaultdict(list)
    rutas[inicio] = [[inicio]]

    # Cola de prioridad
    cola = [(0, 0, inicio)]

    while cola:
        dist_actual, arcos_actual, nodo_actual = heapq.heappop(cola)

        if nodo_actual == fin:
            continue

        for vecino, peso in grafo[nodo_actual].items():
            distancia = dist_actual + peso
            num_arcos = arcos_actual + 1

            # Si encontramos un camino mejor
            if distancia < distancias[vecino]:
                distancias[vecino] = distancia
                arcos[vecino] = num_arcos
                rutas[vecino] = [ruta + [vecino] for ruta in rutas[nodo_actual]]
                heapq.heappush(cola, (distancia, num_arcos, vecino))

            # Si encontramos un camino con la misma distancia pero menos arcos
            elif distancia == distancias[vecino] and num_arcos < arcos[vecino]:
                arcos[vecino] = num_arcos
                rutas[vecino] = [ruta + [vecino] for ruta in rutas[nodo_actual]]
                heapq.heappush(cola, (distancia, num_arcos, vecino))

            # Si encontramos un camino con la misma distancia y mismos arcos
            elif distancia == distancias[vecino] and num_arcos == arcos[vecino]:
                rutas[vecino].extend([ruta + [vecino] for ruta in rutas[nodo_actual]])

    # Eliminamos rutas duplicadas
    rutas_unicas = []
    for ruta in rutas[fin]:
        if ruta not in rutas_unicas:
            rutas_unicas.append(ruta)

    return distancias[fin], rutas_unicas

# Ejemplo de uso
grafo = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

peso_total, caminos = dijkstra_con_rutas(grafo, 'A', 'D')
print(f"Peso total mínimo: {peso_total}")
print("Rutas óptimas:")
for camino in caminos:
    print(" -> ".join(camino))

Peso total mínimo: 4
Rutas óptimas:
A -> B -> C -> D
